<a href="https://colab.research.google.com/github/arif481/Final_Year_Project/blob/main/QRNG_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit qiskit-aer matplotlib
import time
import secrets
import requests
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

# --- 1. Local Quantum Simulation (Qiskit) ---
class QuantumSimulatorRNG:
    def __init__(self):
        self.backend = AerSimulator()

    def generate_bits(self, num_bits):
        # Original implementation generates 'num_bits' single bits and concatenates.
        # If generate_bits(10) is called, it will run 10 shots of a 1-qubit circuit,
        # yielding 10 individual bits. This is correct for generating a 10-bit string.

        qc = QuantumCircuit(1, 1)
        qc.h(0)
        qc.measure(0, 0)
        job = self.backend.run(qc, shots=num_bits, memory=True) # Each shot yields one bit
        result = job.result()
        memory = result.get_memory()
        return "".join(memory)

# --- 2. Remote True Quantum (ANU API) - WITH RETRY LOGIC (kept for reference, not used in benchmark below) ---
class AnuWebQRNG:
    def __init__(self):
        self.url = "https://qrng.anu.edu.au/API/jsonI.php"
        self.max_retries = 3
        self.retry_delay = 1  # seconds

    def generate_bits(self, num_bits):
        num_bytes = (num_bits // 8) + (1 if num_bits % 8 != 0 else 0)
        params = {
            "length": num_bytes,
            "type": "uint8"
        }

        for attempt in range(self.max_retries):
            try:
                response = requests.get(self.url, params=params, timeout=10)
                response.raise_for_status()
                data = response.json()["data"]
                binary_string = "".join(f"{x:08b}" for x in data)
                return binary_string[:num_bits]

            except requests.exceptions.HTTPError as e:
                if response.status_code == 500:
                    print(f"  Attempt {attempt + 1}/{self.max_retries}: Server error, retrying...")
                    if attempt < self.max_retries - 1:
                        time.sleep(self.retry_delay)
                    else:
                        return f"Error: {e} (max retries exceeded)"
                else:
                    return f"Error: {e}"
            except Exception as e:
                return f"Error: {e}"

        return "Error: Failed after all retries"

# --- 2.1 New Remote QRNG (Random.org Byte Stream API) ---
class RandomOrgWebQRNG:
    def __init__(self):
        self.url = "https://www.random.org/cgi-bin/randbyte"
        self.max_retries = 3
        self.retry_delay = 1  # seconds

    def generate_bits(self, num_bits):
        num_bytes_needed = (num_bits // 8) + (1 if num_bits % 8 != 0 else 0)
        params = {
            "num": num_bytes_needed,
            "format": "hex"
        }

        for attempt in range(self.max_retries):
            try:
                response = requests.get(self.url, params=params, timeout=10)
                response.raise_for_status() # Raise an exception for HTTP errors
                hex_string = response.text.strip() # Response is plain hex text

                # Remove spaces and newlines from the hex string before conversion
                hex_string_cleaned = hex_string.replace(' ', '').replace('\n', '')

                # Handle cases where hex_string_cleaned might be empty
                if not hex_string_cleaned:
                    raise ValueError("Received empty hexadecimal string from API")

                binary_string = bin(int(hex_string_cleaned, 16))[2:].zfill(num_bytes_needed * 8)

                return binary_string[:num_bits]

            except requests.exceptions.HTTPError as e:
                # Specific handling for HTTP errors
                print(f"  Random.org Attempt {attempt + 1}/{self.max_retries}: HTTP Error {e.response.status_code}, retrying...")
                if attempt < self.max_retries - 1:
                    time.sleep(self.retry_delay)
                else:
                    return f"Error: HTTP Error {e.response.status_code} (max retries exceeded)"
            except requests.exceptions.ConnectionError as e:
                # Handle connection errors
                print(f"  Random.org Attempt {attempt + 1}/{self.max_retries}: Connection Error: {e}, retrying...")
                if attempt < self.max_retries - 1:
                    time.sleep(self.retry_delay)
                else:
                    return f"Error: Connection Error (max retries exceeded)"
            except requests.exceptions.Timeout as e:
                # Handle timeout errors
                print(f"  Random.org Attempt {attempt + 1}/{self.max_retries}: Timeout Error: {e}, retrying...")
                if attempt < self.max_retries - 1:
                    time.sleep(self.retry_delay)
                else:
                    return f"Error: Timeout Error (max retries exceeded)"
            except Exception as e:
                # Catch any other unexpected errors
                print(f"  Random.org Attempt {attempt + 1}/{self.max_retries}: Unexpected Error: {e}, retrying...")
                if attempt < self.max_retries - 1:
                    time.sleep(self.retry_delay)
                else:
                    return f"Error: Unexpected Error {e} (max retries exceeded)"

        return "Error: Random.org failed after all retries"


# --- 3. Classical Baseline (Python Secrets) ---
class ClassicalRNG:
    def __init__(self):
        pass # No initialization needed for secrets module

    def generate_bits(self, num_bits):
        num_bytes = (num_bits // 8) + (1 if num_bits % 8 != 0 else 0)
        random_bytes = secrets.token_bytes(num_bytes)

        binary_string = "".join(f"{x:08b}" for x in random_bytes)
        return binary_string[:num_bits]

# --- 4. Benchmarking Engine (for general bit generation) ---
def run_benchmark(target_bits=1024):
    generators = {
        "Local Qiskit Sim": QuantumSimulatorRNG(),
        "Classical OS": ClassicalRNG(),
        "Remote Random.org API": RandomOrgWebQRNG() # Using Random.org for benchmark
    }

    results = {}

    print(f"\n--- Benchmarking Generation of {target_bits} bits ---")
    print(f"{'Method':<25} | {'Time (sec)':<10} | {'Speed (bits/s)':<15}")
    print("-" * 60)

    for name, gen in generators.items():
        start_time = time.perf_counter()
        bits = gen.generate_bits(target_bits)
        end_time = time.perf_counter()

        duration = end_time - start_time

        if "Error" in bits:
            print(f"{name:<25} | FAILED     | {bits}")
            continue

        speed = target_bits / duration
        results[name] = duration

        print(f"{name:<25} | {duration:.5f} s  | {speed:.0f}")

    return results

# --- 5. Visualization (for general bit generation benchmark) ---
def plot_results(results):
    names = list(results.keys())
    times = list(results.values())

    plt.figure(figsize=(10, 6))
    bars = plt.bar(names, times, color=['#6929c4', '#009d9a', '#1192e8'])

    plt.ylabel('Time to Generate (Seconds)')
    plt.title('QRNG Simulation Time Comparison')
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, f'{yval:.4f}s', va='bottom', ha='center')

    plt.show()


# --- New function: Generate and Benchmark 3-Digit Numbers ---
def generate_and_benchmark_3digit_numbers(num_required=30, min_val=100, max_val=999, max_attempts_per_num=50):
    generators = {
        "Local Qiskit Sim": QuantumSimulatorRNG(),
        "Classical OS": ClassicalRNG(),
        "Remote Random.org API": RandomOrgWebQRNG() # Using Random.org for 3-digit number benchmark
    }

    benchmark_results = {}

    print(f"\n--- Benchmarking Generation of {num_required} 3-digit numbers ({min_val}-{max_val}) ---")

    for name, gen in generators.items():
        start_time = time.perf_counter()
        collected_numbers = []
        attempts = 0

        print(f"Generating for {name}...")

        # Each 3-digit number (100-999) fits within 10 bits (since 2^9=512, 2^10=1024)
        # So we need to request at least 10 bits.
        bits_to_request = 10

        while len(collected_numbers) < num_required and attempts < num_required * max_attempts_per_num:
            bits_string = gen.generate_bits(bits_to_request)
            attempts += 1

            if "Error" in bits_string:
                print(f"  {name}: Encountered error: {bits_string}. Stopping generation for this method.")
                collected_numbers = [] # Clear any partially collected numbers if an error occurs
                break

            try:
                integer_val = int(bits_string, 2)
            except ValueError:
                # This can happen if bits_string is malformed for some reason
                continue

            if min_val <= integer_val <= max_val:
                collected_numbers.append(integer_val)

        end_time = time.perf_counter()
        duration = end_time - start_time

        benchmark_results[name] = {
            "time": duration,
            "numbers": collected_numbers,
            "num_collected": len(collected_numbers),
            "total_attempts": attempts
        }

        if len(collected_numbers) < num_required:
            print(f"  {name}: Only collected {len(collected_numbers)} out of {num_required} numbers after {attempts} attempts.")
        else:
            print(f"  {name}: Successfully collected {len(collected_numbers)} numbers in {duration:.4f}s after {attempts} attempts.")

    return benchmark_results

# Call the new function for 30 numbers
benchmark_3digit_data = generate_and_benchmark_3digit_numbers(num_required=30)

# Optional: Print the collected data for inspection (30 numbers)
print("\n--- Detailed 3-Digit Number Benchmark Results ---")
for method, data in benchmark_3digit_data.items():
    print(f"\nMethod: {method}")
    print(f"  Time taken: {data['time']:.4f}s")
    print(f"  Numbers collected: {data['num_collected']}/{30}")
    print(f"  Total attempts: {data['total_attempts']}")
    # print(f"  Generated numbers: {data['numbers']}") # Uncomment to see actual numbers

# Call the new function for 60 numbers
benchmark_60_3digit_data = generate_and_benchmark_3digit_numbers(num_required=60)

# Optional: Print the collected data for inspection (60 numbers)
print("\n--- Detailed 3-Digit Number Benchmark Results (60 numbers) ---")
for method, data in benchmark_60_3digit_data.items():
    print(f"\nMethod: {method}")
    print(f"  Time taken: {data['time']:.4f}s")
    print(f"  Numbers collected: {data['num_collected']}/{60}")
    print(f"  Total attempts: {data['total_attempts']}")

# Call the new function for 90 numbers
benchmark_90_3digit_data = generate_and_benchmark_3digit_numbers(num_required=90)

# Optional: Print the collected data for inspection (90 numbers)
print("\n--- Detailed 3-Digit Number Benchmark Results (90 numbers) ---")
for method, data in benchmark_90_3digit_data.items():
    print(f"\nMethod: {method}")
    print(f"  Time taken: {data['time']:.4f}s")
    print(f"  Numbers collected: {data['num_collected']}/{90}")
    print(f"  Total attempts: {data['total_attempts']}")


--- Benchmarking Generation of 30 3-digit numbers (100-999) ---
Generating for Local Qiskit Sim...
  Local Qiskit Sim: Successfully collected 30 numbers in 0.0282s after 37 attempts.
Generating for Classical OS...
  Classical OS: Successfully collected 30 numbers in 0.0001s after 37 attempts.
Generating for Remote Random.org API...
  Remote Random.org API: Successfully collected 30 numbers in 11.2959s after 31 attempts.

--- Detailed 3-Digit Number Benchmark Results ---

Method: Local Qiskit Sim
  Time taken: 0.0282s
  Numbers collected: 30/30
  Total attempts: 37

Method: Classical OS
  Time taken: 0.0001s
  Numbers collected: 30/30
  Total attempts: 37

Method: Remote Random.org API
  Time taken: 11.2959s
  Numbers collected: 30/30
  Total attempts: 31

--- Benchmarking Generation of 60 3-digit numbers (100-999) ---
Generating for Local Qiskit Sim...
  Local Qiskit Sim: Successfully collected 60 numbers in 0.0491s after 65 attempts.
Generating for Classical OS...
  Classical OS: Suc

In [ ]:
print("\n--- Summary of 3-Digit Number Generation and Benchmarking ---")
for method, data in benchmark_3digit_data.items():
    print(f"\nMethod: {method}")
    print(f"  Time taken: {data['time']:.4f} seconds")
    print(f"  Numbers collected: {data['num_collected']}/30")
    if data['num_collected'] > 0:
        # Print the numbers in a more readable format, possibly wrapped or limited if too long
        print(f"  Generated numbers: {data['numbers']}")
    else:
        print("  No numbers were successfully generated.")
    if 'Error' in str(data.get('numbers', '')) or data['num_collected'] < 30:
        print(f"  Status: FAILED or Incomplete (Attempts: {data['total_attempts']})")
    else:
        print(f"  Status: Success (Total attempts: {data['total_attempts']})")



--- Summary of 3-Digit Number Generation and Benchmarking ---

Method: Local Qiskit Sim
  Time taken: 0.0282 seconds
  Numbers collected: 30/30
  Generated numbers: [726, 842, 449, 829, 907, 772, 160, 534, 575, 549, 126, 333, 147, 795, 700, 676, 176, 545, 717, 860, 390, 377, 953, 719, 444, 335, 224, 631, 992, 241]
  Status: Success (Total attempts: 37)

Method: Classical OS
  Time taken: 0.0001 seconds
  Numbers collected: 30/30
  Generated numbers: [594, 876, 136, 485, 770, 403, 323, 546, 829, 606, 540, 597, 918, 893, 764, 540, 599, 819, 594, 289, 451, 430, 890, 516, 516, 414, 279, 358, 863, 355]
  Status: Success (Total attempts: 37)

Method: Remote Random.org API
  Time taken: 11.2959 seconds
  Numbers collected: 30/30
  Generated numbers: [815, 514, 990, 651, 900, 927, 941, 702, 667, 960, 787, 567, 929, 853, 663, 866, 770, 530, 986, 631, 729, 825, 597, 685, 905, 952, 753, 762, 792, 688]
  Status: Success (Total attempts: 31)


60 3-Digit Number Generation and Benchmarking

In [ ]:
print("\n--- Summary of 60 3-Digit Number Generation and Benchmarking ---")
for method, data in benchmark_60_3digit_data.items():
    print(f"\nMethod: {method}")
    print(f"  Time taken: {data['time']:.4f} seconds")
    print(f"  Numbers collected: {data['num_collected']}/60")
    if data['num_collected'] > 0:
        print(f"  Generated numbers: {data['numbers']}")
    else:
        print("  No numbers were successfully generated.")
    if 'Error' in str(data.get('numbers', '')) or data['num_collected'] < 60:
        print(f"  Status: FAILED or Incomplete (Attempts: {data['total_attempts']})")
    else:
        print(f"  Status: Success (Total attempts: {data['total_attempts']})")


--- Summary of 60 3-Digit Number Generation and Benchmarking ---

Method: Local Qiskit Sim
  Time taken: 0.0491 seconds
  Numbers collected: 60/60
  Generated numbers: [882, 592, 869, 284, 983, 313, 792, 291, 760, 362, 667, 184, 522, 335, 829, 986, 401, 545, 882, 964, 373, 854, 781, 288, 921, 492, 706, 905, 531, 743, 801, 957, 757, 135, 113, 989, 385, 732, 960, 645, 191, 920, 384, 763, 509, 895, 978, 770, 757, 188, 821, 294, 632, 305, 507, 437, 598, 862, 586, 249]
  Status: Success (Total attempts: 65)

Method: Classical OS
  Time taken: 0.0002 seconds
  Numbers collected: 60/60
  Generated numbers: [210, 966, 407, 885, 504, 645, 585, 426, 583, 541, 904, 142, 934, 510, 243, 282, 356, 727, 846, 869, 458, 639, 687, 733, 700, 203, 930, 729, 805, 590, 821, 574, 246, 644, 516, 630, 631, 723, 988, 725, 592, 181, 922, 670, 799, 819, 349, 182, 495, 773, 607, 631, 737, 826, 154, 660, 174, 620, 119, 784]
  Status: Success (Total attempts: 71)

Method: Remote Random.org API
  Time taken: 24.7660

### Benchmarking Generation of 90 3-Digit Numbers

In [ ]:
print("\n--- Summary of 90 3-Digit Number Generation and Benchmarking ---")
for method, data in benchmark_90_3digit_data.items():
    print(f"\nMethod: {method}")
    print(f"  Time taken: {data['time']:.4f} seconds")
    print(f"  Numbers collected: {data['num_collected']}/90")
    if data['num_collected'] > 0:
        print(f"  Generated numbers: {data['numbers']}")
    else:
        print("  No numbers were successfully generated.")
    if 'Error' in str(data.get('numbers', '')) or data['num_collected'] < 90:
        print(f"  Status: FAILED or Incomplete (Attempts: {data['total_attempts']})")
    else:
        print(f"  Status: Success (Total attempts: {data['total_attempts']})")


--- Summary of 90 3-Digit Number Generation and Benchmarking ---

Method: Local Qiskit Sim
  Time taken: 0.4132 seconds
  Numbers collected: 90/90
  Generated numbers: [812, 227, 640, 853, 626, 944, 377, 785, 777, 481, 675, 421, 526, 731, 593, 759, 867, 509, 237, 160, 780, 334, 331, 498, 161, 651, 841, 173, 158, 757, 286, 785, 402, 498, 137, 277, 630, 891, 380, 802, 882, 528, 917, 192, 583, 787, 198, 611, 937, 462, 981, 386, 309, 405, 762, 754, 853, 337, 873, 444, 297, 581, 396, 736, 376, 270, 365, 192, 948, 890, 185, 412, 391, 551, 264, 331, 920, 860, 392, 169, 462, 240, 774, 748, 164, 746, 959, 651, 973, 775]
  Status: Success (Total attempts: 110)

Method: Classical OS
  Time taken: 0.0003 seconds
  Numbers collected: 90/90
  Generated numbers: [696, 118, 483, 879, 168, 143, 289, 271, 753, 212, 296, 103, 924, 521, 177, 844, 278, 411, 745, 194, 342, 181, 432, 638, 905, 688, 696, 214, 730, 850, 182, 143, 300, 505, 212, 224, 850, 389, 425, 566, 212, 766, 545, 849, 942, 920, 665, 131, 